## Setup steps
- Add TAVILY_API_KEY to `.env` file. You can get this from [Tavily AI](https://app.tavily.com/home).
- Add Anthropic api key to `.env` file

In [5]:
# Import relevant functionality
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

from dotenv import load_dotenv

load_dotenv()

# Create the agent
memory = SqliteSaver.from_conn_string(":memory:")
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Bob! Since you didn't ask a specific question, I don't need to use any of the tools for this response. It's nice to meet you. San Francisco is a beautiful city with lots to see and do. What are some of your favorite things about living there?", response_metadata={'id': 'msg_01KcV9rcG2aiU6vuB8i9Ne8K', 'model': 'claude-3-sonnet-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 271, 'output_tokens': 60}}, id='run-b1fee727-0fca-46b3-8951-02ef2856d448-0', usage_metadata={'input_tokens': 271, 'output_tokens': 60, 'total_tokens': 331})]}}
----
{'agent': {'messages': [AIMessage(content=[{'text': 'To get the current weather for your location in San Francisco, I can use the search tool:', 'type': 'text'}, {'id': 'toolu_015zyugt7KpkgAXzxEhtBNcU', 'input': {'query': 'san francisco weather'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_01Er7iqC5bpPaMwEHNrQdMrK', 'model

# Define Tools
We first need to create the tools we want to use. Our main tool of choice will be Tavily - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1722617601, 'localtime': '2024-08-02 9:53'}, 'current': {'last_updated_epoch': 1722617100, 'last_updated': '2024-08-02 09:45', 'temp_c': 16.1, 'temp_f': 61.0, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 244, 'wind_dir': 'WSW', 'pressure_mb': 1019.0, 'pressure_in': 30.08, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 80, 'cloud': 10, 'feelslike_c': 16.1, 'feelslike_f': 61.0, 'windchill_c': 16.1, 'windchill_f': 61.0, 'heatindex_c': 16.1, 'heatindex_f': 61.0, 'dewpoint_c': 12.6, 'dewpoint_f': 54.6, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 5.0, 'gust_mph': 5.6, 'gust_kph': 9.1}}"}, {'url': 'https://www.timeanddate.com/weather/usa/san-fran

# Using Language Models
Next, let's learn how to use a language model by to call tools. LangChain supports many different language models that you can use interchangably - select the one you want to use below!

In [7]:
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(model="claude-3-5-sonnet-20240620")

As per previous tutorials, you can call the model by passing in a list of messages.

In [8]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today? Feel free to ask me any questions or let me know if you need help with anything.'

We can now see what it is like to enable this model to do tool calling. In order to enable that we use .bind_tools to give the language model knowledge of these tools

In [9]:
model_with_tools = model.bind_tools(tools) # this tells the model it can use the Tavily search tool (2 results) as defined above.

We can now call the model. Let's first call it with a normal message, and see how it responds. We can look at both the content field as well as the tool_calls field.

In [10]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! Welcome. How can I assist you today? I'm here to help with any questions or tasks you might have.
ToolCalls: []


As expected, there were no tool calls. Now lets try calling it with some input that would expect a tool to be called.

In [15]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF? Use tools available.")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: [{'text': "I apologize, but I don't have a specific weather tool available to check the current weather in San Francisco. However, I can use the search function to find the most up-to-date weather information for you. Let me do that now.", 'type': 'text'}, {'id': 'toolu_011vNREzEgms8FDPcr3MfFGA', 'input': {'query': 'current weather in San Francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}]
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'toolu_011vNREzEgms8FDPcr3MfFGA', 'type': 'tool_call'}]


NOTE: This needed a bit of an extra prompt to trigger the tool call.

After a tweak to the prompt, we can see that there is a tool call. This isn't calling the ool yet. To use the tool, we will need to create an agent.

# Create the agent.
Now that we have defined the tools and the LLM, we can create the agent. We will be using LangGraph to construct the agent. Currently we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.

Now, we can initialize the agent with the LLM and the tools.

Note that we are passing in the `model`, not `model_with_tools`. That is because `create_react_agent` will call `.bind_tools` for us under the hood.

In [16]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

# Run the agent
We can now run the agent on a few queries! Note that for now, these are all stateless queries (it won't remember previous interactions). Note that the agent will return the final state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

First up, let's see how it responds when there's no need to call a tool:

In [17]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='7e7edc75-0083-4278-98b4-a870f756426e'),
 AIMessage(content="Hello! Welcome to our conversation. How can I assist you today? I'm here to help with a wide range of topics and questions. Is there anything specific you'd like to know or discuss?", response_metadata={'id': 'msg_01DCGT3fzXG9QvchkC7W1oBA', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 399, 'output_tokens': 43}}, id='run-a37b100c-39b3-4138-8eca-e1c7dbd0576d-0', usage_metadata={'input_tokens': 399, 'output_tokens': 43, 'total_tokens': 442})]

Let's now try it out on an example where it should be invoking the tool

In [20]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf? Use the TavilySearchResults tool.")]} # needed to add the second sentence to give the agent a nudget to use the tool.
)
response["messages"]

[HumanMessage(content='whats the weather in sf? Use the TavilySearchResults tool.', id='49527414-2fbf-4b86-bbb7-3811bf38294e'),
 AIMessage(content=[{'text': "I apologize, but I don't have a specific tool for weather information. However, I can use the Tavily search engine to find current weather information for San Francisco. Let me do that for you.", 'type': 'text'}, {'id': 'toolu_01P8yhjjCf9gWmjnYakREEc4', 'input': {'query': 'current weather in San Francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_01Doa6qgr1MmEtJVdufSeP9d', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 414, 'output_tokens': 106}}, id='run-6c366004-92db-4e31-ba45-163f5a9b7be4-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'toolu_01P8yhjjCf9gWmjnYakREEc4', 'type': 'tool_call'}], usage_metadata={'input_tokens': 414, 'output_tokens':

# Streaming Messages
We've seen how the agent can be called with `.invoke` to get back a final response. If the agent is executing multiple steps, that may take a while. In order to show intermediate progress, we can stream back messages as they occur.

In [21]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf? Use the TavilySearchResults tool.")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content=[{'text': "I apologize, but I don't have a specific tool for checking the current weather. However, I can use the Tavily search tool to find recent weather information for San Francisco. Let me do that for you.", 'type': 'text'}, {'id': 'toolu_01KC995U11fCLfDehbnigMQe', 'input': {'query': 'current weather in San Francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_01ERp7PGwGmcaMgUwXxhH5qB', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 414, 'output_tokens': 107}}, id='run-23efe035-def3-4527-b993-9f2a954ca7fd-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'toolu_01KC995U11fCLfDehbnigMQe', 'type': 'tool_call'}], usage_metadata={'input_tokens': 414, 'output_tokens': 107, 'total_tokens': 521})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://

# Streaming Tokens
In addition to streaming back messages, it is also useful tob e streaming back tokens. We can do this with the `.astream_events` method.

In [22]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

/Users/errollee/GitHub/langchain_tutorial_python/venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


[{'text': 'I', 'type': 'text', 'index': 0}]|[{'text': ' apolog', 'type': 'text', 'index': 0}]|[{'text': 'ize, but I', 'type': 'text', 'index': 0}]|[{'text': ' don', 'type': 'text', 'index': 0}]|[{'text': "'t have", 'type': 'text', 'index': 0}]|[{'text': ' direct', 'type': 'text', 'index': 0}]|[{'text': ' access to real-', 'type': 'text', 'index': 0}]|[{'text': 'time weather information', 'type': 'text', 'index': 0}]|[{'text': '. However', 'type': 'text', 'index': 0}]|[{'text': ', I can help', 'type': 'text', 'index': 0}]|[{'text': ' you find the', 'type': 'text', 'index': 0}]|[{'text': ' current weather in', 'type': 'text', 'index': 0}]|[{'text': ' San Francisco using', 'type': 'text', 'index': 0}]|[{'text': ' a', 'type': 'text', 'index': 0}]|[{'text': ' search tool', 'type': 'text', 'index': 0}]|[{'text': '.', 'type': 'text', 'index': 0}]|[{'text': ' Let', 'type': 'text', 'index': 0}]|[{'text': ' me ', 'type': 'text', 'index': 0}]|[{'text': 'do that', 'type': 'text', 'index': 0}]|[{'t

# Adding in memory
As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a `thread_id` when invoking the agent (so it knows which thread/conversation to resume from).

In [23]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Bob! It's nice to meet you. How can I assist you today? Is there any particular information you're looking for or any questions you have? I'm here to help with a wide range of topics and can use various tools to find accurate and up-to-date information for you if needed.", response_metadata={'id': 'msg_011h5USKpZVJz5hLiMm3Jegx', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 401, 'output_tokens': 66}}, id='run-3e41d767-acf7-4271-91b1-75e031f63373-0', usage_metadata={'input_tokens': 401, 'output_tokens': 66, 'total_tokens': 467})]}}
----


In [24]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob. You introduced yourself as "bob" in your first message.', response_metadata={'id': 'msg_018ghWCu8bp1L3qTNKzG1UBH', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 475, 'output_tokens': 20}}, id='run-2f69eb94-4501-47ed-b3da-93ce490c5ea4-0', usage_metadata={'input_tokens': 475, 'output_tokens': 20, 'total_tokens': 495})]}}
----


In [25]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I apologize, but I don't have access to personal information about individual users, including names. As an AI assistant, I don't have any prior knowledge about you or your personal details. Is there something else I can help you with?", response_metadata={'id': 'msg_015VtPspmBFwSb2kA5WPy7Gm', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'input_tokens': 402, 'output_tokens': 52}}, id='run-5d02a989-d19d-4282-8ff0-a95ab51919e3-0', usage_metadata={'input_tokens': 402, 'output_tokens': 52, 'total_tokens': 454})]}}
----
